In [ ]:
import os
import numpy as np
import cv2

root_dir = 'D:\\Alphabet'
dest_dir = 'D:\\frames'

if not os.path.exists(dest_dir):
    os.mkdir(dest_dir)

# To list what are the directories - train, test
data_dir_list = os.listdir(root_dir)

def vid_to_frames():
    for data_dir in data_dir_list: # read the train  directory one 
        data_path = os.path.join(root_dir,data_dir) # 'Alphabet/train'
        dest_data_path = os.path.join(dest_dir,data_dir) # 'frames/train'
        if not os.path.exists(dest_data_path):
            os.mkdir(dest_data_path)
        
        activity_list = os.listdir(data_path) # there are  individual  letters directories ['A', 'Ą', 'B'.....] etc
        
        for activity in activity_list: # loop over every letter folder
            activity_path = os.path.join(data_path,activity) # 'Alphabet/train/A'
            dest_activity_path = os.path.join(dest_data_path,activity) # 'frames/train/A'
            if not os.path.exists(dest_activity_path):
                os.mkdir(dest_activity_path)
            write_frames(activity_path,dest_activity_path)
    
def write_frames(activity_path,dest_activity_path):
    # read the list of video from 'Alphabet/train/A' - [A1.mp4,A2.mp4, ......]
    vid_list = os.listdir(activity_path) 
    for vid in vid_list: # A1.mp4
        dest_folder_name = vid[:-4] # v_Archery_g01_c01
        dest_folder_path = os.path.join(dest_activity_path,dest_folder_name) # 'frames/train/A/A1'
        if not os.path.exists(dest_folder_path):
            os.mkdir(dest_folder_path)
            
        vid_path = os.path.join(activity_path,vid)  # 'Alphabet/train/A/A1.mp4'
        print ('video path: ', vid_path)
        cap = cv2.VideoCapture(vid_path) # initialize a cap object for reading the video
        
        ret=True
        frame_num=0
        while ret:
            ret, img = cap.read()
            output_file_name = 'img_{:06d}'.format(frame_num) + '.png' # img_000001.png
            # output frame to write 'frames/train/A/A1/img_000001.png'
            output_file_path = os.path.join(dest_folder_path, output_file_name)
            frame_num += 1
            #print("Frame no. ", frame_num)
            try:
                #cv2.imshow('img',img)
                cv2.waitKey(5)
                cv2.imwrite(output_file_path, img) # writing frames to defined location
            except Exception as e:
                print(e)
            if ret==False:
                cv2.destroyAllWindows()
                cap.release()
if __name__ == '__main__':
    vid_to_frames()

In [ ]:
import cv2
import math
import numpy as np
DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480


def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  return img

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img=resize_and_show(img)
            images.append(img)
    return images


images=load_images_from_folder("C:\\Users\\HP\\Desktop\\video_to_frames\\frames\\train\\B\\1 (3)")

In [ ]:
images

In [ ]:
import mediapipe as mp
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
# Run MediaPipe Hands.
output_path='C:\\Users\\HP\\Desktop\\video_to_frames\\output\\B'
flag=0
counter=0
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.7) as hands:
  for image in images:
    if(flag==1):
      cv2.destroyAllWindows()
      break
    # Convert the BGR image to RGB, flip the image around y-axis for correct 
    # handedness output and process it with MediaPipe Hands.
    results = hands.process(cv2.flip(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 1))

    # Print handedness (left v.s. right hand).
    #print(f'Handedness of {name}:')
    print(results.multi_handedness)

    if not results.multi_hand_landmarks:
      continue
    # Draw hand landmarks of each hand.
    #print(f'Hand landmarks of {name}:')
    image_hight, image_width, _ = image.shape
    annotated_image = cv2.flip(image.copy(), 1)
    for hand_landmarks in results.multi_hand_landmarks:
      # Print wrist finger tip coordinates.
      if(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y<0.4):
        counter+=1
        print(
            f'wrist tip coordinate: (',
            f'{hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x }, '
            f'{hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y })'
        )
        mp_drawing.draw_landmarks(
            annotated_image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        img=resize_and_show(cv2.flip(annotated_image, 1))
        cv2.imwrite(os.path.join(output_path,f'B{counter}.png') ,img)
        #cv2.imshow('img',img)
        if cv2.waitKey(5) & 0xFF == 27:
          flag=1
          break
